<a href="https://colab.research.google.com/github/XaxUser/Projects/blob/main/smolagents_doc/en/rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
! pip install git+https://github.com/huggingface/smolagents.git

  Cloning https://github.com/huggingface/smolagents.git to /tmp/pip-req-build-7h499hdi
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/smolagents.git /tmp/pip-req-build-7h499hdi
  Resolved https://github.com/huggingface/smolagents.git to commit 7113cf6b3987c7f93ea17f9963fc32fd6cee2846
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [9]:
! pip install smolagents pandas langchain langchain-community sentence-transformers datasets python-dotenv rank_bm25 --upgrade

In [10]:
! pip install langchain langchain_google_genai

## Building an Agentic RAG System

In [11]:
from smolagents import Tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.retrievers import BM25Retriever
import datasets
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import userdata
import google.generativeai as genai
import os
from dotenv import load_dotenv

In [12]:
hf_token = userdata.get('HF_TOKEN')
os.environ["HF_TOKEN"] = hf_token

In [13]:
api_key = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = api_key
genai.configure(api_key=api_key)

In [14]:
load_dotenv()

False

In [15]:
# Load the Hugging Face documentation dataset
knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

# Filter to include only Transformers documentation
knowledge_base = knowledge_base.filter(lambda row: row["source"].startswith("huggingface/transformers"))

# Convert dataset entries to Document objects with metadata
source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

# Split documents into smaller chunks for better retrieval
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Characters per chunk
    chunk_overlap=50,  # Overlap between chunks to maintain context
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],  # Priority order for splitting
)
docs_processed = text_splitter.split_documents(source_docs)

print(f"Knowledge base prepared with {len(docs_processed)} document chunks")

Knowledge base prepared with 14695 document chunks


In [16]:
class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        # Initialize the retriever with our processed documents
        self.retriever = BM25Retriever.from_documents(
            docs, k=10  # Return top 10 most relevant documents
        )

    def forward(self, query: str) -> str:
        """Execute the retrieval based on the provided query."""
        assert isinstance(query, str), "Your search query must be a string"

        # Retrieve relevant documents
        docs = self.retriever.invoke(query)

        # Format the retrieved documents for readability
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

# Initialize our retriever tool with the processed documents
retriever_tool = RetrieverTool(docs_processed)

In [17]:
# Define the ChatGoogleGenerativeAI model
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # You can specify the model ID here
    verbose=True  # Enable verbose to display detailed output
)

# Define a prompt template for your question
prompt_template = "Answer the following query based on the retrieved documents: {query}"
prompt = PromptTemplate(input_variables=["query"], template=prompt_template)

# Set up an LLM chain to connect the model with the retriever
llm_chain = LLMChain(llm=llm, prompt=prompt)

<ipython-input-17-202164313>:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)


In [18]:
# Function to use retriever and then pass the result to ChatGoogleGenerativeAI for final response
def ask_question_with_retriever(question: str):
    # Step 1: Retrieve relevant documents using the retriever tool
    print("Step 1: Retrieving relevant documents based on the query...")
    retrieved_docs = retriever_tool.forward(question)
    print(retrieved_docs)  # Show the retrieved documents

    # Step 2: Pass the retrieved documents to the LLM chain for final response
    print("Step 2: Generating the final answer using the model...")
    response = llm_chain.run({"query": question + "\n" + retrieved_docs})

    return response

In [19]:
# Ask a question that requires retrieving information
question = "For a transformers model training, which is slower, the forward or the backward pass?"

# Get the answer from the agent
agent_output = ask_question_with_retriever(question)

# Display the final answer
print("\nFinal answer:")
print(agent_output)

<ipython-input-18-3511770212>:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_chain.run({"query": question + "\n" + retrieved_docs})


Step 1: Retrieving relevant documents based on the query...

Retrieved documents:


===== Document 0 =====
For convolutions and linear layers there are 2x flops in the backward compared to the forward, which generally translates 
into ~2x slower (sometimes more, because sizes in the backward tend to be more awkward). Activations are usually 
bandwidth-limited, and it’s typical for an activation to have to read more data in the backward than in the forward 
(e.g. activation forward reads once, writes once, activation backward reads twice, gradOutput and output of the forward,

===== Document 1 =====
overhead. This is super helpful when you have activation checkpointing enabled, where we do a forward recompute and
backward passes a single layer granularity and want to keep the parameter in the forward recompute till the backward

===== Document 2 =====
- A train step function which combines the loss function and optimizer update, does the forward and backward pass and returns the updated